In [11]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import time
import itertools
from itertools import product



options = {
 "WLSACCESSID":"a4353fb7-f95b-4075-b288-ca3f60983b36",
"WLSSECRET":"d894d460-2dac-4210-8c40-c91c68ecfb13",
"LICENSEID":2562382
}

In [24]:
K = 3
lambda_star = np.array([0,.2,3])

I = 2
J = 5
# sigma = 1

### generate exogenous data

# np.random.seed(2)
# φ_i_j_k = np.random.normal(0,1, size=[I,J,K])
# # eps_ij = sigma * np.random.normal(0,1, size = [I,J])
# def φ_k(i_idx,bundles):
#     return (φ_i_j_k[i_idx[:,None], bundles,:]).sum(1)



np.random.seed(4)
φ_i_j_k = np.random.normal(0,5, size=[I,J,K-1])
# eps_i_j = sigma * np.random.normal(0,1, size = [I,J])

phi_j = np.random.normal(0,1, size=[J])**2
t_j_j = np.random.normal(0,1, size=[J,J])**2
t_j_sum = t_j_j.sum(1)
def φ_k(i_idx,bundles):
    complementarities_i_k = (phi_j[bundles] * (t_j_j[bundles[:,:,None] , bundles[:,None,:]]).sum(-1) 
                             / t_j_sum[bundles]).sum(-1)
    
    return np.hstack(((φ_i_j_k[i_idx[:,None], bundles,:]).sum(1), complementarities_i_k.reshape(-1, 1) ))

In [25]:
φ_k(np.array([0]), np.array([[0,1]], dtype = int))

array([[-4.72673612,  5.96774921,  0.33237689]])

In [26]:
def grad_lovatz_extension(i_idx, z_i_j, lambda_k, eps_i_j = None) :
    if eps_i_j is None:
        eps_i_j = np.zeros([len(i_idx),J])              
    # tic = time.time()
    sorted_z_id_j = z_i_j.argsort(1)[:,::-1]
    # toc =time.time()
    # print(f"sorting time {toc-tic}")


    # tic = time.time()
    grad = np.zeros([len(z_i_j),J])
    for j in range(J):
        grad[np.arange(len(i_idx)),sorted_z_id_j[:,j]] = (φ_k(i_idx, sorted_z_id_j[:,np.arange(j+1)] ) @ lambda_k 
                                    - φ_k(i_idx, sorted_z_id_j[:,np.arange(j)] ) @ lambda_k  
                                    + eps_i_j[np.arange(len(i_idx)),sorted_z_id_j[:,j]])

    # toc = time.time()
    # print(f"grad: {toc - tic} seconds")

    return grad


def maximize_supermodular(i_idx, lambda_k , num_iterations,alpha ,eps_i_j = None):

    z_t = np.ones([len(i_idx),J])/2 

    z_list = []
    iter = 0 
    for _ in range(num_iterations):
        grad = grad_lovatz_extension(i_idx, z_t, lambda_k, eps_i_j) 
        z_new = z_t + alpha * grad / np.linalg.norm(grad)
        z_new = np.clip(z_new, 0, 1)

        z_t = z_new
        z_list.append(z_new)
        iter += 1

    z_star = np.array(z_list)[-int(np.floor(num_iterations)/10):,:,:].mean(0)
    bundle_star = np.array(z_star.round(0), dtype= bool)
    print('violations of rounding in SFM: ', len(np.where((z_star > 0.2) & (z_star < 0.8))[0]))

    return bundle_star, z_star

In [27]:
num_iterations = 400
alpha = 0.1
tic = time.time()
# Run maximize_supermodular for all indices in parallel
i_idx = np.arange(I)
# B_star_tensor, z_star_tensor = maximize_supermodular(i_idx, lambda_star, num_iterations, alpha)
toc = time.time()
print(f"Time taken: {toc - tic} seconds")

Time taken: 0.00018596649169921875 seconds


In [28]:
i_idx = np.arange(I)
z_t = np.random.rand(I,J)
tic = time.time()
grad = grad_lovatz_extension(i_idx, z_t, lambda_star, eps_i_j = None) 
toc = time.time()
print(f"Time taken: {toc - tic} seconds")

Time taken: 0.00131988525390625 seconds


In [29]:
# def maximize_naive(i_idx, lambda_k  ,eps_i_j = None):
#     if eps_i_j is None:
#         eps_i_j = np.zeros([len(i_idx),J]) 
    
#     bundle_star = (φ_i_j_k[i_idx] @ lambda_star + eps_i_j) > 1e-13
#     print( np.max(φ_i_j_k[i_idx] @ lambda_star + eps_i_j))
#     return bundle_star

In [55]:
max_iters = 8
tol = 1e-12


constraints_list = []

# Create the environment with license parameters
with gp.Env(params=options) as env:
    # Create a Gurobi model within the environment
    with gp.Model(env=env) as model:
        ### Initialize 
        # Create variables
        u_i = model.addVars(I, lb= 0 ,ub= GRB.INFINITY, name="utilities")
        p_j = model.addVars(J, lb= 0 ,ub= GRB.INFINITY, name="prices")

        model.setObjective( u_i.sum() + p_j.sum(), GRB.MINIMIZE)
        # Optimize the model
        model.setParam('OutputFlag', 0)
        model.optimize()
        theta_solution = np.array(model.x)

        ### Column Generation
        iter = 0
        while iter < max_iters:
            print("##############")
            print(f"ITER: {iter}")
            ### Pricing problem
          
            B_star_i, _ = maximize_supermodular(np.arange(I), lambda_star, 300, 1,
                                                - np.ones(I)[:,None] * theta_solution[None, I:]) 
            print(B_star_i*1)
            reduced_cost_i = []
            for i in range(I):
                  reduced_cost_i.append(φ_k(np.array([i]), np.where(B_star_i[i])[0].reshape(1,-1))[0] @ lambda_star 
                                - theta_solution[i] - theta_solution[I + np.where(B_star_i[i])[0] ].sum() )
            reduced_cost_i = np.array(reduced_cost_i)

            # stop if certificate holds
            print(f"reduced cost: {np.max(reduced_cost_i)}")
            if np.max(reduced_cost_i) <= tol:
                primal_solution = np.array(model.x)
                dual_solution = np.array(model.pi)
                print("DONE!")
                break
            
            ### Master problem

            # model.addConstrs((u_i[i] + gp.quicksum(p_j[j] for j in np.where(B_star_i[i])[0])>= 
            #                 φ_k(np.array([i]), np.where(B_star_i[i])[0].reshape(1,-1))[0] @ lambda_star 
            #                 for i in range(I)), name="constraint_batch")
            
            model.addConstr(u_i[0] >= 5)
   
            
            print([φ_k(np.array([i]), np.where(B_star_i[i])[0].reshape(1, -1))[0] @ lambda_star for i in range(I)])
            print('constraints: ', model.NumConstrs)
       
        

            print( [ φ_k(np.array([i]), np.where(B_star_i[i])[0].reshape(1,-1))[0] @ lambda_star  for i in range(I)])
            print( 'constraints: ', model.NumConstrs)

       
            # stop for 3 seconds
            time.sleep(3)
            constraints_list.append(B_star_i)
            print('utilities: ', theta_solution)
            print('value of LP: ', model.ObjVal)

            # Optimize the model
            u_i.start = reduced_cost_i + theta_solution[:I]
            p_j.start = theta_solution[-J:]
            model.optimize()
            theta_solution = np.array(model.x)
            iter += 1
            
            

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2562382
Academic license 2562382 - for non-commercial use only - registered to ed___@nyu.edu
##############
ITER: 0
violations of rounding in SFM:  0
[[1 1 1 1 1]
 [1 1 1 1 1]]
reduced cost: 24.387440976484942
[21.259133857563373, 24.387440976484942]
constraints:  0
[21.259133857563373, 24.387440976484942]
constraints:  0
utilities:  [0. 0. 0. 0. 0. 0. 0.]
value of LP:  0.0
##############
ITER: 1
violations of rounding in SFM:  0
[[1 1 1 1 1]
 [1 1 1 1 1]]
reduced cost: 24.387440976484942
[21.259133857563373, 24.387440976484942]
constraints:  1
[21.259133857563373, 24.387440976484942]
constraints:  1
utilities:  [5. 0. 0. 0. 0. 0. 0.]
value of LP:  5.0
##############
ITER: 2
violations of rounding in SFM:  0
[[1 1 1 1 1]
 [1 1 1 1 1]]
reduced cost: 24.387440976484942
[21.259133857563373, 24.387440976484942]
constraints:  2
[21.259133857563373, 24.387440976484942]
constraints:  2
utilities:  [5. 0. 0. 0. 

In [31]:
theta_solution

array([0.        , 3.12830712, 7.77523494, 1.19285725, 8.58321771,
       1.29936872, 2.40845524])

In [32]:
binding_constraints = np.array(constraints_list).reshape(-1,J)[dual_solution >0]
index = np.kron(np.ones(iter, dtype= int),np.arange(I))
matching = np.zeros((I ,J), dtype=bool)
matching[index[dual_solution > 0]]= binding_constraints

In [33]:
matching.sum(0)

array([1, 1, 1, 1, 1])

In [34]:
matching

array([[False, False, False, False, False],
       [ True,  True,  True,  True,  True]])